In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os

In [2]:
path = r'C:\Users\shahe\Desktop\Instacart Basket Analysis'

In [3]:
df_ords_prods_merge = pd.read_pickle(r'C:\Users\shahe\Desktop\Instacart Basket Analysis\Data\Prepared data\df_ords_prods_merge.pkl')

# Step 2

In [4]:
# Groupping by department ID and aggregating the order number by its Mean 

df_ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


# Step 3

In the sample there are less department id, if we talk about the whole data set there are all the departments id that is 1 to 21

# Step 4

In [5]:
# Groupping by user ID and Aggergating the max number with order number 

df_ords_prods_merge['max_order'] = df_ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [6]:
df_ords_prods_merge.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,2539329,1,prior,1,2,8,NaN,196.0,1.0,0.0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Most orders,10
1,2398795,1,prior,2,3,7,15.0,196.0,1.0,1.0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest busy,Average orders,10
2,473747,1,prior,3,3,12,21.0,196.0,1.0,1.0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest busy,Most orders,10
3,2254736,1,prior,4,4,7,29.0,196.0,1.0,1.0,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest busy,Average orders,10
4,431534,1,prior,5,4,15,28.0,196.0,1.0,1.0,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest busy,Most orders,10


In [7]:
# flagging by loyalty

df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df_ords_prods_merge.loc[(df_ords_prods_merge['max_order'] <= 40) & (df_ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [8]:
# Loyalty flags values

df_ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

# Step 5

In [9]:
# Loyalty flags aggregated by prices

df_ords_prods_merge.groupby('loyalty_flag').agg({'prices' : ['mean','max','min','sum']})

prices                           
                       mean      max  min          sum
loyalty_flag                                          
Loyal customer    10.386336  99999.0  1.0  106814042.2
New customer      13.294670  99999.0  1.0   83011787.2
Regular customer  12.495717  99999.0  1.0  198391693.2

Loyal customer buying the less and the new customer are buying more 

# Step 6

In [10]:
# Groupping by user ID and Aggergating the max number with order number 

df_ords_prods_merge['avg_price'] = df_ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [11]:
# flagging by loyalty

df_ords_prods_merge.loc[df_ords_prods_merge['avg_price'] < 10, 'spending_flag'] = 'Low Spender'
df_ords_prods_merge.loc[df_ords_prods_merge['avg_price'] >= 10 , 'spending_flag'] = 'High Spender'

In [12]:
df_ords_prods_merge.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag
0,2539329,1,prior,1,2,8,NaN,196.0,1.0,0.0,...,9.0,both,Mid-range product,Regularly busy,Regularly busy,Most orders,10,New customer,6.367797,Low Spender
1,2398795,1,prior,2,3,7,15.0,196.0,1.0,1.0,...,9.0,both,Mid-range product,Regularly busy,Slowest busy,Average orders,10,New customer,6.367797,Low Spender
2,473747,1,prior,3,3,12,21.0,196.0,1.0,1.0,...,9.0,both,Mid-range product,Regularly busy,Slowest busy,Most orders,10,New customer,6.367797,Low Spender
3,2254736,1,prior,4,4,7,29.0,196.0,1.0,1.0,...,9.0,both,Mid-range product,Least busy,Slowest busy,Average orders,10,New customer,6.367797,Low Spender
4,431534,1,prior,5,4,15,28.0,196.0,1.0,1.0,...,9.0,both,Mid-range product,Least busy,Slowest busy,Most orders,10,New customer,6.367797,Low Spender


In [13]:
# Spending frequency

df_ords_prods_merge['spending_flag'].value_counts()

Low Spender     31770614
High Spender      634245
Name: spending_flag, dtype: int64

# step 7

In [14]:
# Getting the median of days_since_prior_order based on user ID

df_ords_prods_merge['days_order'] = df_ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [15]:
# flagging by frequency

df_ords_prods_merge.loc[df_ords_prods_merge['days_order'] > 20, 'frequent_flag'] = 'Non-frequent customer'
df_ords_prods_merge.loc[(df_ords_prods_merge['days_order'] > 10) & (df_ords_prods_merge['days_order'] <= 20), 'frequency_flag'] = 'Regular customer'
df_ords_prods_merge.loc[df_ords_prods_merge['days_order'] <= 10, 'frequent_flag'] = 'Frequent customer'

In [16]:
df_ords_prods_merge['frequent_flag'].value_counts()

Frequent customer        21559853
Non-frequent customer     3636437
Name: frequent_flag, dtype: int64

# Step 9

In [17]:
# Export data to pkl file

df_ords_prods_merge.to_pickle(os.path.join(path, 'Data','Prepared Data', 'ords_prods_merge.pkl'))